In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext rpy2.ipython

In [157]:
import numpy as np
import pandas as pd

from kaggle.house_prices import helpers
from kaggle.house_prices import missing
from kaggle.house_prices import outliers

combined_dataset = helpers.load_data()
combined_dataset = missing.fix_all(combined_dataset)
combined_dataset = outliers.remove_outliers(combined_dataset)
combined_dataset['price_log'] = np.log(combined_dataset['SalePrice'])
combined_dataset = combined_dataset.drop(['SalePrice', 'Id'], axis=1)
combined_dataset.shape

(2917, 83)

In [158]:
from kaggle.house_prices import quantile_rating as QR

ratings = QR.calc_ratings(
    df=combined_dataset, 
    target_var='price_log', 
    rating_quantiles=QR.calc_quantiles(
        sample=combined_dataset['price_log'],
        probs=[0.25, 0.5, 0.75]
    ), 
    categ_vars=helpers.get_character_colnames(combined_dataset)
)

ratings.head(7)

,var,value,rating
0,Alley,Grvl,1.500000
1,Alley,Pave,2.536585
2,Alley,_none_,2.531822
3,BldgType,1Fam,2.564039
4,BldgType,2fmCon,1.580645
5,BldgType,Duplex,1.692308
6,BldgType,Twnhs,1.883721


In [117]:
default_rating = ratings[ratings['var'].isna()]['rating'].iloc[0]
default_rating

2.5

In [118]:
step1 = pd.melt(
    frame=combined_dataset[helpers.get_character_colnames(combined_dataset)], 
    var_name='var', 
    value_name='value'
)

step1.head()

,var,value
0,Alley,_none_
1,Alley,_none_
2,Alley,_none_
3,Alley,_none_
4,Alley,_none_


In [119]:
step2 = (
    step1
    .join(
        ratings.set_index(['var', 'value']), 
        on=['var', 'value'], 
        how='left'
    )
    .fillna({'rating': default_rating})
)

step2.head()

len(step2[step2['rating'].isna()])

,var,value,rating
0,Alley,_none_,2.531822
1,Alley,_none_,2.531822
2,Alley,_none_,2.531822
3,Alley,_none_,2.531822
4,Alley,_none_,2.531822


0

In [171]:
step3 = (
    step2
    [['var', 'rating']]
    .assign(
        id=lambda df: df.groupby('var').cumcount()
    )
    .set_index(['var', 'id'])
    .unstack(0)
)
step3.columns = step3.columns.get_level_values(1)
step3.columns.name = None
step3.index.name = None
step3 = step3.reset_index(drop=True)
step3.head()

,Alley,BldgType,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtQual,CentralAir,Condition1,Condition2,...,MoSold,Neighborhood,PavedDrive,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType,Street,Utilities
0,2.531822,2.564039,2.540871,2.320042,3.262019,2.553429,3.066343,2.57887,2.547619,2.50277,...,2.538462,2.966667,2.584454,2.491736,2.483601,2.418054,2.451586,2.421468,2.499311,2.496911
1,2.531822,2.564039,2.540871,3.295455,2.231818,2.553429,3.066343,2.57887,1.987500,2.50277,...,2.382353,3.454545,2.584454,2.491736,2.483601,2.418054,2.451586,2.421468,2.499311,2.496911
2,2.531822,2.564039,2.540871,2.675439,3.262019,2.553429,3.066343,2.57887,2.547619,2.50277,...,2.698413,2.966667,2.584454,2.491736,2.483601,2.418054,2.451586,2.421468,2.499311,2.496911
3,2.531822,2.564039,3.046154,2.320042,2.231818,2.553429,1.847458,2.57887,2.547619,2.50277,...,2.538462,3.156863,2.584454,2.491736,2.483601,2.418054,1.920792,2.421468,2.499311,2.496911
4,2.531822,2.564039,2.540871,2.900452,3.262019,2.553429,3.066343,2.57887,2.547619,2.50277,...,2.711864,3.975610,2.584454,2.491736,2.483601,2.418054,2.451586,2.421468,2.499311,2.496911


In [172]:
rest = combined_dataset.drop(columns=helpers.get_character_colnames(combined_dataset))

rest.shape
step3.shape

rest.index
step3.index


pd.concat([step3, rest], axis=1).head()


(2917, 38)

(2917, 45)

RangeIndex(start=0, stop=2917, step=1)

RangeIndex(start=0, stop=2917, step=1)

,Alley,BldgType,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtQual,CentralAir,Condition1,Condition2,...,PoolArea,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,dataSource,price_log
0,2.531822,2.564039,2.540871,2.320042,3.262019,2.553429,3.066343,2.57887,2.547619,2.50277,...,0,0,8,856.0,0,2003,2003,2008,train,12.247694
1,2.531822,2.564039,2.540871,3.295455,2.231818,2.553429,3.066343,2.57887,1.987500,2.50277,...,0,0,6,1262.0,298,1976,1976,2007,train,12.109011
2,2.531822,2.564039,2.540871,2.675439,3.262019,2.553429,3.066343,2.57887,2.547619,2.50277,...,0,0,6,920.0,0,2001,2002,2008,train,12.317167
3,2.531822,2.564039,3.046154,2.320042,2.231818,2.553429,1.847458,2.57887,2.547619,2.50277,...,0,0,7,756.0,0,1915,1970,2006,train,11.849398
4,2.531822,2.564039,2.540871,2.900452,3.262019,2.553429,3.066343,2.57887,2.547619,2.50277,...,0,0,9,1145.0,192,2000,2000,2008,train,12.429216
